# Get the data from the Database and format into Matrices

In [2]:
import pandas as pd
from retry import retry
from tqdm import tqdm
import psycopg2 as pg
from sqlalchemy import create_engine 
import sys
sys.path.insert(1, r"")
import cred2 as cred

# create engine
username = cred.user
password = cred.password
host = cred.host
database = cred.dbname
port = str(cred.port)
engine = create_engine('postgresql://'+username+':'+password+'@'+host+':'+port+'/'+database)

# Functions

In [3]:
def create_tweets_mat(keywords):
    ''' 
    This function selects the aggregated tweets per county and topic from the database and concats them over time.
    Finally, the tweets are stored in matrix form. (dates, countyid)
    '''
    years_list = ['2020', '2021', '2022']
    dfs = []

    for year in years_list:
        query = f"SELECT date, countyfips, {keywords} FROM covid.tweets_{year}_agg_county"
        df = pd.read_sql(query, con=engine)
        dfs.append(df)

    # Concatenate the DataFrames for all years
    df_concat = pd.concat(dfs, ignore_index=True)

    pivot_table = df_concat.pivot_table(index='date', columns='countyfips', values=keywords, aggfunc='sum')
    pivot_table.to_csv('../../../data/tweet_mats/tweets_'+keywords+'.csv')


# Concat tweet tables 

In [4]:
keywords_list = column_names = [
    'keywords_covid_baseline_topic',
    'all_tweets'
] # this workflow can be extended to fitt several different keyword lists!


for keywords in keywords_list:

    df = create_tweets_mat(keywords)
    
    